In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import re
# Set the environment variable to use only the GPU with ID 1 (GTX 1080 Ti)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import cv2
import os
import PIL
import csv
# import tensorflow as tf

# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential 
import pandas as pd
import numpy as np
import pathlib

In [11]:
import shutil

def delete_directory(path):
    try:
        shutil.rmtree(path)
        print(f"Directory '{path}' and all its contents have been deleted successfully.")
    except FileNotFoundError:
        print(f"The directory '{path}' does not exist.")
    except PermissionError:
        print(f"Permission denied: Unable to delete '{path}'.")
    except Exception as e:
        print(f"Error: {e}")

In [14]:
test_data_dir='/home/rag-tt/tactile5/tactile_images/1'
delete_directory(test_data_dir)

Directory '/home/rag-tt/tactile5/tactile_images/1' and all its contents have been deleted successfully.


In [2]:
data_dir='/home/rag-tt/tactile6/tactile_images/'
data_dir= pathlib.Path(data_dir)
# Construct the absolute path to the image file
image_path = os.path.join(data_dir, '5', '0.jpg')
image_path

'/home/rag-tt/tactile6/tactile_images/5/0.jpg'

In [3]:
image = cv2.imread(image_path)
print(image.shape)
# plt.imshow(image)
# plt.axis('off')  # Hide the axis
# plt.show()

(480, 640, 3)


In [4]:
def find_no_of_images(obj_id):
    image_dir = os.path.join(data_dir, str(obj_id))
    image_dir= pathlib.Path(image_dir)
    no_of_images= len(list(image_dir.glob('*.jpg')))
    return no_of_images

In [5]:
no_of_images= find_no_of_images(6)
no_of_images

97

In [6]:
def delete_images_and_labels(obj_id, img_ids):
    for img_id in img_ids:
        tactile_image_path = os.path.join(data_dir, str(obj_id), str(img_id)+ '.jpg')
        depth_image_path = os.path.join(data_dir, str(obj_id),'depth'+ str(img_id)+ '.png')
        # Check if the file exists
        if os.path.exists(tactile_image_path) and os.path.exists(depth_image_path) :
            # Delete the file
            os.remove(tactile_image_path)
            os.remove(depth_image_path)
            print(f"Image at",tactile_image_path, "has been deleted.")
            print(f"Image at",depth_image_path, "has been deleted.")
        else:
            print(f"No file found at",tactile_image_path, "or")
            print(f"No file found at",depth_image_path)
            
            
        csv_path = os.path.join(data_dir, str(obj_id),'slip_log.csv')
        
        # Read all rows from the CSV file
        with open(csv_path, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            csv_data = list(reader)
        csv_img_ids= [row[0] for row in csv_data]
        
        
        #find the line number in csv file to delete
        index = 0
        for csv_img_id in csv_img_ids:
            if csv_img_id == str(img_id):
                line_number = index
                break
            else:
                line_number = None
            index += 1
        print(line_number)    
        
        
        # Check if the line number is valid
        if line_number is not None:
            # Delete the specified line (0-indexed)
            del csv_data[line_number]
            
            # Write the updated rows back to the CSV file
            with open(csv_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(csv_data)

            print(f"Deleted line with image_id {img_id} and saved the updated CSV.")
        else:
            print(f"Line with image id {img_id} not found.")

In [7]:
def fill_image_sequence(folder_path):
    files = os.listdir(folder_path)
    # Filter files by the specified extension
    tactile_images = [f for f in files if f.endswith('.jpg')]
    tactile_images.sort(key=lambda f: int(re.search(r'\d+', f).group()))
    depth_images = [f for f in files if f.endswith('.png')]
    depth_images.sort(key=lambda f: int(re.search(r'\d+', f).group()))
    
    img_id = 0
    for tactile_image, depth_image in zip(tactile_images,depth_images): 
        old_tactile_image_path = os.path.join(folder_path,tactile_image)
        old_depth_image_path = os.path.join(folder_path,depth_image)
        new_tactile_image_path = os.path.join(folder_path,str(img_id) + '.jpg')
        new_depth_image_path = os.path.join(folder_path, 'depth'+str(img_id)+'.png')
        os.rename(old_tactile_image_path, new_tactile_image_path)
        os.rename(old_depth_image_path, new_depth_image_path)
        img_id = img_id +1



In [8]:
path = os.path.join(data_dir, str(33))
fill_image_sequence(path)
    

In [9]:
def remove_blank_images(data_dir, no_of_samples = 70):
    image_paths = []
    
    for obj_id in range(no_of_samples):
        no_of_images = find_no_of_images(obj_id)
        if no_of_images < 40:
            continue
        if no_of_images > 300:
            object_folder_path = os.path.join(data_dir, str(obj_id))
            delete_directory(object_folder_path)
            continue
            
        
        csv_path = os.path.join(data_dir, str(obj_id),'slip_log.csv')
        label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
        previous_highest_pixel_value = 0
        blank_image_ids = []
        blank_image_spotted = 0
        
        
        for img_id in range(no_of_images):
            
            # import the images as numpy arrays
            tactile_image_path = os.path.join(data_dir, str(obj_id), str(img_id)+ '.jpg')
            depth_image_path = os.path.join(data_dir, str(obj_id),'depth'+ str(img_id)+ '.png')
            if not os.path.exists(tactile_image_path):
                continue
            image = Image.open(depth_image_path)
            image_array = np.array(image)
            
            
            # Find the maximum value in the array
            current_highest_pixel_value = np.max(image_array)
            
            # delete the blank images based on the previous and next maximum values in array
            # images with sudden unexpeted changes along with their labels will be deleted
            if current_highest_pixel_value < 0.2*previous_highest_pixel_value:
                blank_image_ids.append(img_id)
                blank_image_spotted = 1
            else:
                if blank_image_spotted == 1:
                    delete_images_and_labels(obj_id, blank_image_ids)
                    blank_image_ids=[]
                    blank_image_spotted =0
                previous_highest_pixel_value = current_highest_pixel_value
        if len(blank_image_ids) > 3:
            blank_image_ids = blank_image_ids[3:]
            print(blank_image_ids)
            delete_images_and_labels(obj_id, blank_image_ids)
        #rename the images to fill the gaps        
        path = os.path.join(data_dir, str(obj_id))
        fill_image_sequence(path)
                

In [10]:
remove_blank_images(data_dir, no_of_samples=2279)

Image at /home/rag-tt/tactile6/tactile_images/8/73.jpg has been deleted.
Image at /home/rag-tt/tactile6/tactile_images/8/depth73.png has been deleted.
73
Deleted line with image_id 73 and saved the updated CSV.
Image at /home/rag-tt/tactile6/tactile_images/8/74.jpg has been deleted.
Image at /home/rag-tt/tactile6/tactile_images/8/depth74.png has been deleted.
73
Deleted line with image_id 74 and saved the updated CSV.
Image at /home/rag-tt/tactile6/tactile_images/8/76.jpg has been deleted.
Image at /home/rag-tt/tactile6/tactile_images/8/depth76.png has been deleted.
74
Deleted line with image_id 76 and saved the updated CSV.
[81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]
Image at /home/rag-tt/tactile6/tactile_images/8/81.jpg has been deleted.
Image at /home/rag-tt/tactile6/tactile_images/8/depth81.png has been deleted.
78
Deleted line with image_id 81 and saved the updated CSV.
Image at /home/rag-tt/tactile6/tactile_images/8/82.jpg has been deleted.
Image at /

FileNotFoundError: /home/rag-tt/tactile6/tactile_images/1053/slip_log.csv not found.